In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn as nn
import torch


In [ ]:
df = pd.read_csv("/content/Dataset for Detection of Cyber-Aggression - Dataset for Detection of Cyber-Aggression.csv")

In [ ]:
df.head()

,Text,label
0,Get fucking real dude.,AG
1,She is as dirty as they come and that crook R...,AG
2,why did you fuck it up. I could do it all day ...,AG
3,Dude they dont finish enclosing the fucking sh...,AG
4,WTF are you talking about Men? No men thats no...,AG


In [ ]:
# Split data into train, validation, and test sets (80-10-10 split)
train_texts, test_texts, train_labels, test_labels = train_test_split(df["Text"], df["label"], test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=42)


In [ ]:
label_mapping = {'AG': 0, 'NAG': 1}  # Define a mapping from label strings to integers
train_labels = train_labels.map(label_mapping)
val_labels = val_labels.map(label_mapping)
test_labels = test_labels.map(label_mapping)

In [ ]:
# Choose the tokenizer and model for BERT
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
model_bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Choose the tokenizer and model for RoBERTa
tokenizer_roberta = RobertaTokenizer.from_pretrained("roberta-base")
model_roberta = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize and encode the data for BERT
train_encodings_bert = tokenizer_bert(train_texts.tolist(), truncation=True, padding=True)
val_encodings_bert = tokenizer_bert(val_texts.tolist(), truncation=True, padding=True)
test_encodings_bert = tokenizer_bert(test_texts.tolist(), truncation=True, padding=True)


In [ ]:
# Tokenize and encode the data for RoBERTa
train_encodings_roberta = tokenizer_roberta(train_texts.tolist(), truncation=True, padding=True)
val_encodings_roberta = tokenizer_roberta(val_texts.tolist(), truncation=True, padding=True)
test_encodings_roberta = tokenizer_roberta(test_texts.tolist(), truncation=True, padding=True)

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
# Create datasets for BERT
train_dataset_bert = CustomDataset(train_encodings_bert, train_labels.tolist())
val_dataset_bert = CustomDataset(val_encodings_bert, val_labels.tolist())
test_dataset_bert = CustomDataset(test_encodings_bert, test_labels.tolist())


In [ ]:
# Create datasets for RoBERTa
train_dataset_roberta = CustomDataset(train_encodings_roberta, train_labels.tolist())
val_dataset_roberta = CustomDataset(val_encodings_roberta, val_labels.tolist())
test_dataset_roberta = CustomDataset(test_encodings_roberta, test_labels.tolist())


In [ ]:
# Define the training parameters
num_epochs = 3
batch_size_train = 8
learning_rate_model = 2e-5


In [ ]:
# Define the loss function and optimizer for BERT
loss_function_bert = nn.CrossEntropyLoss()
optimizer_model_bert = optim.AdamW(model_bert.parameters(), lr=learning_rate_model)


In [ ]:
# Define the loss function and optimizer for RoBERTa
loss_function_roberta = nn.CrossEntropyLoss()
optimizer_model_roberta = optim.AdamW(model_roberta.parameters(), lr=learning_rate_model)


In [ ]:
# Move model to the appropriate device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_bert.to(device)
model_roberta.to(device)


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
def train_bert_early_stopping(model_bert, optimizer_model_bert, train_dataset_bert, validation_dataset_bert, epochs, batch_size_train, early_stopping_patience=3):

  best_validation_loss = float('inf')
  epochs_no_improvement = 0

  for epoch in range(epochs):
    model_bert.train()
    train_loader_bert = DataLoader(train_dataset_bert, batch_size=batch_size_train, shuffle=True)

    # Training loop (omitted for brevity)

    validation_loss = evaluate(model_bert, validation_dataset_bert)
    if validation_loss < best_validation_loss:
      best_validation_loss = validation_loss
      epochs_no_improvement = 0
    else:
      epochs_no_improvement += 1

    if epochs_no_improvement >= early_stopping_patience:
      print(f"Early stopping triggered after {epochs_no_improvement} epochs with no improvement")
      break

  print(f"Finished training for {epoch + 1} epochs")


In [ ]:
def train_roberta_early_stopping(model_roberta, optimizer_roberta, train_dataset_roberta, validation_dataset_roberta, epochs, batch_size_train, early_stopping_patience=3):

  best_validation_loss = float('inf')
  epochs_no_improvement = 0

  for epoch in range(epochs):
    model_roberta.train()
    train_loader_roberta = DataLoader(train_dataset_roberta, batch_size=batch_size_train, shuffle=True)

    for batch in train_loader_roberta:
      optimizer_roberta.zero_grad()
      input_ids = batch['input_ids'].to(torch.device('cuda')) if torch.cuda.is_available() else batch['input_ids']
      attention_mask = batch['attention_mask'].to(torch.device('cuda')) if torch.cuda.is_available() else batch['attention_mask']
      labels = batch['labels'].to(torch.device('cuda')) if torch.cuda.is_available() else batch['labels']
      outputs = model_roberta(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss
      loss.backward()
      optimizer_roberta.step()

    # Early stopping
    validation_loss = evaluate(model_roberta, validation_dataset_roberta)  # Implement your evaluation function
    if validation_loss < best_validation_loss:
      best_validation_loss = validation_loss
      epochs_no_improvement = 0
    else:
      epochs_no_improvement += 1

    if epochs_no_improvement >= early_stopping_patience:
      print(f"Early stopping triggered after {epochs_no_improvement} epochs with no improvement")
      break

  print(f"Finished training for {epoch + 1} epochs")


In [ ]:
# Evaluation function
def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(torch.device('cuda')) if torch.cuda.is_available() else batch['input_ids']
            attention_mask = batch['attention_mask'].to(torch.device('cuda')) if torch.cuda.is_available() else batch['attention_mask']
            labels = batch['labels'].to(torch.device('cuda')) if torch.cuda.is_available() else batch['labels']
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, axis=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    return accuracy, precision, recall, f1



In [ ]:
# Evaluation on BERT model
batch_size = 32  # Replace 32 with the desired batch size
test_loader_bert = DataLoader(test_dataset_bert, batch_size=batch_size)
accuracy_bert, precision_bert, recall_bert, f1_bert = evaluate(model_bert, test_loader_bert)
print("Evaluation metrics for BERT:")
print(f"Accuracy: {accuracy_bert}, Precision: {precision_bert}, Recall: {recall_bert}, F1 Score: {f1_bert}")


Evaluation metrics for BERT:
Accuracy: 0.41179410294852575, Precision: 0.0, Recall: 0.0, F1 Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Evaluation on RoBERTa model
batch_size = 32
test_loader_roberta = DataLoader(test_dataset_roberta, batch_size=batch_size)
accuracy_roberta, precision_roberta, recall_roberta, f1_roberta = evaluate(model_roberta, test_loader_roberta)
print("Evaluation metrics for RoBERTa:")
print(f"Accuracy: {accuracy_roberta}, Precision: {precision_roberta}, Recall: {recall_roberta}, F1 Score: {f1_roberta}")


Evaluation metrics for RoBERTa:
Accuracy: 0.5882058970514743, Precision: 0.5882058970514743, Recall: 1.0, F1 Score: 0.7407174323473883
